# Actor Critic with replay memory

In [1]:
import gym
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import random
from IPython.display import clear_output
from itertools import count
from logger import Logger
from pyvirtualdisplay import Display
from collections import namedtuple

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
from torch.autograd import Variable

In [2]:
virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [3]:
############ HYPERPARAMETERS ##############
FRAMES = 2
RESIZE_PIXELS = 40

device = 'cuda:3'

POLICY_LR = 5e-6
Q_LR = 1e-3
GAMMA = 0.99
BETA = 0.05
END_SCORE = 1000
MEMORY_SIZE = 150000
BATCH_SIZE = 128
############################################

In [4]:
############ ENVIRONMENT ##############

class Environment:
    def __init__(self):
        self.env = gym.make("CartPole-v0").unwrapped
        self.env.reset()
        
        screen = self.env.render(mode='rgb_array').transpose((2,0,1))
        _, self.screen_height, self.screen_width = screen.shape
        
        self.resize = T.Compose([T.ToPILImage(),
                    T.Resize(RESIZE_PIXELS, interpolation=Image.CUBIC),
                    T.Grayscale(),
                    T.ToTensor()])
        
        world_width = self.env.x_threshold * 2
        self.scale = self.screen_width / world_width
        
    def get_cart_location(self):
        return int(self.env.state[0] * self.scale + self.screen_width / 2.0)
        
    def get_screen(self):
        screen = self.env.render(mode='rgb_array').transpose((2, 0, 1))
        
        screen = screen[:, int(self.screen_height*0.4):int(self.screen_height * 0.8)]
        screen = np.ascontiguousarray(screen, dtype=np.float32) / 255
        screen = torch.from_numpy(screen)
        return self.resize(screen).unsqueeze(0)
    
    def close(self):
        self.env.close()
        
    def step(self, action):
        return self.env.step(action)
    
    def reset(self):
        self.env.reset()

In [5]:
############ MEMORY ##############

Transition = namedtuple('Transition',('state', 'action', 'next_state', 'reward'))

class ReplayMemory(object):

    def __init__(self, capacity,memory=None):
        self.capacity = capacity
        if memory != None:
            self.memory = memory
        else:
            self.memory = []
            
        self.position = 0

    def push(self, *args):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [6]:
############ NETWORK ##############
'''
For policy gradient, the network should input the raw pixels, and output the probabilities of choosing either 0 or 1
'''

class PolicyNetwork(nn.Module):
    def __init__(self,h=60,w=135,device='cuda:0'):
        super().__init__()
        
        self.device = device
        
        input_channel = 2
        hidden_channel = 32#64
        hidden_channel2 = 32
        kernel_size = 5
        stride = 2
        
        self.base = nn.Sequential(
            nn.Conv2d(input_channel, hidden_channel, kernel_size = kernel_size, stride = stride),
            nn.BatchNorm2d(hidden_channel),
            nn.ReLU(),
            nn.Conv2d(hidden_channel, hidden_channel, kernel_size = kernel_size, stride = stride),
            nn.BatchNorm2d(hidden_channel),
            nn.ReLU(),
            nn.Conv2d(hidden_channel, hidden_channel2, kernel_size = kernel_size, stride = stride),
            nn.BatchNorm2d(hidden_channel2),
            nn.ReLU()
        )
        
        def conv2d_size_out(size, kernel_size = kernel_size, stride = stride):
            return (size - (kernel_size - 1) - 1) // stride  + 1
        
        convw = conv2d_size_out(conv2d_size_out(conv2d_size_out(w)))
        convh = conv2d_size_out(conv2d_size_out(conv2d_size_out(h)))
        linear_input_size = convw * convh * hidden_channel2
        
        self.head = nn.Sequential(
            nn.Linear(linear_input_size, 24),
            nn.ReLU(),
            nn.Linear(24, 2)
        )

    def forward(self, x):
        #x = x.to(self.device)
        x = self.base(x)
        x = x.flatten(1)
        x = self.head(x)
        return F.softmax(x,dim = 1)
    
class QNetwork(nn.Module):
    def __init__(self,h=60,w=135,device='cuda:0'):
        super().__init__()
        
        self.device = device
        
        input_channel = 2
        hidden_channel = 128#64
        hidden_channel2 = 32
        kernel_size = 5
        stride = 2
        
        self.base = nn.Sequential(
            nn.Conv2d(input_channel, hidden_channel, kernel_size = kernel_size, stride = stride),
            nn.BatchNorm2d(hidden_channel),
            nn.ReLU(),
            nn.Conv2d(hidden_channel, hidden_channel, kernel_size = kernel_size, stride = stride),
            nn.BatchNorm2d(hidden_channel),
            nn.ReLU(),
            nn.Conv2d(hidden_channel, hidden_channel2, kernel_size = kernel_size, stride = stride),
            nn.BatchNorm2d(hidden_channel2),
            nn.ReLU()
        )
        
        def conv2d_size_out(size, kernel_size = kernel_size, stride = stride):
            return (size - (kernel_size - 1) - 1) // stride  + 1
        
        convw = conv2d_size_out(conv2d_size_out(conv2d_size_out(w)))
        convh = conv2d_size_out(conv2d_size_out(conv2d_size_out(h)))
        linear_input_size = convw * convh * hidden_channel2
        
        self.head = nn.Sequential(
            nn.Linear(linear_input_size, 24),
            nn.ReLU(),
            nn.Linear(24, 2)
        )

    def forward(self, x):
        #x = x.to(self.device)
        x = self.base(x)
        x = x.flatten(1)
        x = self.head(x)
        return x

In [7]:
env = Environment()
env.reset()

In [8]:
env.step(0)
plt.figure()
plt.imshow(env.get_screen().cpu().squeeze(0).permute(1, 2, 0).numpy().squeeze(), cmap='gray')
plt.title('Example extracted screen')
plt.show()

In [9]:
screen_shape = env.get_screen().shape
policy_net = PolicyNetwork(h = screen_shape[2], w = screen_shape[3], device = device).to(device)

q_net = QNetwork(h = screen_shape[2], w = screen_shape[3], device = device).to(device)

policy_optimizer = optim.Adam(policy_net.parameters(), lr=POLICY_LR, weight_decay=1e-4)
q_optimizer = optim.Adam(q_net.parameters(), lr = Q_LR, weight_decay = 1e-4)

q_mse_loss = nn.MSELoss()

memory = ReplayMemory(MEMORY_SIZE)

In [10]:
p_screen = env.get_screen().to(device)
c_screen = env.get_screen().to(device)

x = torch.cat((p_screen, c_screen),dim=1)

print("Policy Network output:",policy_net(x))
print("Q Network output:", q_net(x))

Policy Network output: tensor([[0.6050, 0.3950]], device='cuda:1', grad_fn=<SoftmaxBackward>)
Q Network output: tensor([[-0.1001,  0.1641]], device='cuda:1', grad_fn=<AddmmBackward>)


In [11]:
def run_episode(episode, env, logger = None):
    policy_net.eval()
    q_net.eval()
    
    state = env.reset()
    prev_screen = env.get_screen().to(device)
    prev_x = None
    prev_y = None
    
    reward_sum = 0

    for steps in count():
        screen = env.get_screen().to(device)
        
        x = torch.cat((prev_screen, screen), dim=1)
        
        action_prob = policy_net(x)
        action = 0 if random.random() < action_prob[0][0] else 1

        y = torch.tensor([[1, 0]] if action == 0 else [[0, 1]],dtype=torch.int64).to(device)
        
        state, reward, done, _ = env.step(action)
        
        if done:
            x = None
            reward = -30
        
        reward = torch.tensor([reward], dtype=float).to(device)
        
        if prev_x is not None:
            memory.push(prev_x, prev_y, x, reward)
            policy_loss, entropy = update_policy(prev_x, prev_y, q_net, policy_net, policy_optimizer)
            #print(q_net(prev_x).detach(), action_prob)
        
        reward_sum += reward
        prev_screen = screen
        prev_x = x
        prev_y = y
        
        if steps % 10 == 0:
            q_loss = update_q_batch(memory, q_net, policy_net, q_optimizer)
            
        if done or (steps == 1000):
            if logger != None:
                log.log_scalar(scalar=steps,episode=episode,name='duration')
                log.log_scalar(scalar=reward_sum,episode=episode,name='reward')
                log.log_scalar(scalar=entropy,episode=episode,name='entropy')
                log.log_scalar(scalar=q_loss,episode=episode,name='q_loss')
                log.log_scalar(scalar=policy_loss,episode=episode,name='policy_loss')
            print(f"[EPIPSODE] {episode} [DURATION] {steps} [Q_LOSS] {q_loss} [POLICY LOSS] {policy_loss}")
            return False, steps
                
def update_q_batch(memory, q_net, policy_net, optimizer):
    if len(memory) < BATCH_SIZE:
        return 0
    transitions = memory.sample(BATCH_SIZE)
    
    batch = Transition(*zip(*transitions))

    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)
    
    state_action_values = torch.sum(q_net(state_batch) * action_batch,dim=1).to(torch.float64)

    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    
    next_state_values[non_final_mask] = torch.sum(q_net(non_final_next_states).detach() * policy_net(non_final_next_states).detach(),dim=1)
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    loss = F.mse_loss(state_action_values, expected_state_action_values)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    return loss.item()

def update_policy(state, action, q_net, policy_net, optimizer):
    
    # policy_net = policy_net - lr * delta(log(policy_net) * (q_net(s,a) - q_net(s) @ policy_net(s)))
    
    policy_net.train()
    
    action_prob = policy_net(state)
    q = q_net(state).detach()
    
    q_policy = q@action_prob.detach().T
    q_actual = q@action.float().T
    adv = q_actual - q_policy
    
    log_action_prob = torch.log(action_prob)
    log_lik = action.float()@log_action_prob.T
    log_lik_adv = log_lik * adv
    
    entropy = -log_action_prob@action_prob.T
    
    loss = -log_lik_adv - BETA*entropy
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    return loss.item(), entropy.item()

In [12]:
name = f"AC-RM-2 Q_LR={Q_LR} P_LR={POLICY_LR} BATCH_SIZE={BATCH_SIZE} GAMMA={GAMMA} MEMORY_SIZE={MEMORY_SIZE} BETA={BETA}"
log = Logger(model_name=name)

In [ ]:
import time

for episode in count():
    complete, step = run_episode(episode, env, log)

    if complete:
        print('complete...!')
        break
    
    if episode % 50 == 0:
        clear_output(wait=True)
        torch.save(policy_net.state_dict(), f'models/{name}_policy.pt')
        torch.save(q_net.state_dict(), f'models/{name}_q.pt')
        
    if step == 1000:
        torch.save(policy_net.state_dict(), f'models/{name}_policy_{episode}.pt')
        torch.save(q_net.state_dict(), f'models/{name}_q_{episode}.pt')

[EPIPSODE] 301 [DURATION] 11 [Q_LOSS] 16.393779496800963 [POLICY LOSS] -0.015452123247087002
[EPIPSODE] 302 [DURATION] 8 [Q_LOSS] 11.459747293075061 [POLICY LOSS] -0.008439529687166214
[EPIPSODE] 303 [DURATION] 32 [Q_LOSS] 13.312085025966306 [POLICY LOSS] -0.07169903814792633
[EPIPSODE] 304 [DURATION] 16 [Q_LOSS] 16.23813404317645 [POLICY LOSS] 0.008971752598881721
[EPIPSODE] 305 [DURATION] 17 [Q_LOSS] 13.979012517817736 [POLICY LOSS] -0.07436752319335938
[EPIPSODE] 306 [DURATION] 9 [Q_LOSS] 6.976629126363143 [POLICY LOSS] -0.9813740849494934
[EPIPSODE] 307 [DURATION] 32 [Q_LOSS] 13.986532008251297 [POLICY LOSS] -0.08871573954820633
[EPIPSODE] 308 [DURATION] 25 [Q_LOSS] 14.978809371188355 [POLICY LOSS] -0.10688222199678421
[EPIPSODE] 309 [DURATION] 39 [Q_LOSS] 30.346132361334654 [POLICY LOSS] 0.12249387800693512
[EPIPSODE] 310 [DURATION] 12 [Q_LOSS] 11.717418038487443 [POLICY LOSS] 0.4947572350502014
[EPIPSODE] 311 [DURATION] 23 [Q_LOSS] 11.530854182869781 [POLICY LOSS] 0.0260761082172

# 